# 1.**Manage Gpu**


In [1]:
!nvidia-smi

Sat Jun  4 05:07:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **2.connectin Colab and Drive**


In [2]:
from google.colab import drive
drive.mount('/content/gdrive') 
 ##make a shortlink for our address
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 1.png				        Untitled27.ipynb
 2.gdoc				        Untitled28.ipynb
 2.ipynb			        Untitled29.ipynb
 2.png				        Untitled2.ipynb
 3a6ef29e6fdfeeeec8207cd6a8c71414.pdf  'Untitled (3)'
 Book1.xlsx			        Untitled30.ipynb
'Colab Notebooks'		       'Untitled31 (1).ipynb'
'My Drive'			        Untitled31.ipynb
 perceptoron.ipynb		        Untitled32.ipynb
'Project proposal.gdoc'		        Untitled33.ipynb
 svm.ipynb			        Untitled34.ipynb
 tsese.gdoc			        Untitled35.ipynb
 tsese.png			        Untitled36.ipynb
 Untitled			        Untitled37.ipynb
 Untitled0.ipynb		        Untitled38.ipynb
'Untitled (1)'			        Untitled39.ipynb
'Untitled (10)'			        Untitled3.ipynb
 Untitled10.ipynb		       'Untitled (4)'
'Untitled (11)'			        Untitled40.ipynb
 Untitled11.ipynb		        Untitled41.ipynb
'Untitled (12)'			        Untitled42.ipynb
 Untitled12.ipynb		        Untitled43.ipynb
'Untitled (13)'			        Untitled44.ipynb
 Untitled13.ipynb		 

# 3.install and setting Darknet

In [3]:
#clone Darknet
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Total 15420 (delta 0), reused 0 (delta 0), pack-reused 15420
Receiving objects: 100% (15420/15420), 14.05 MiB | 24.39 MiB/s, done.
Resolving deltas: 100% (10360/10360), done.


In [4]:
#configuration Darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:
#compile Darknet
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [6]:
!ls

3rdparty	DarknetConfig.cmake.in	json_mjpeg_streams.sh  scripts
backup		darknet_images.py	LICENSE		       src
build		darknet.py		Makefile	       vcpkg.json
build.ps1	darknet_video.py	net_cam_v3.sh	       video_yolov3.sh
cfg		data			net_cam_v4.sh	       video_yolov4.sh
cmake		image_yolov3.sh		obj
CMakeLists.txt	image_yolov4.sh		README.md
darknet		include			results


## 4.setting yolo

In [7]:
#Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_traning.cfg

In [8]:
#change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_traning.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_traning.cfg
!sed -i 's/max_batches = 500200/max_batches=6000/' cfg/yolov3_traning.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_traning.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_traning.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_traning.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_traning.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_traning.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_traning.cfg



# 5.make file and save result


In [9]:
#create .names files
!echo -e 'Waring Mask\nNot wearing Mask' > data/obj.names


In [10]:
# creat .data files
!echo -e 'classes= 2\ntrain = data/train.txt\n valid = data/test.txt\n names = data/obj.names\n backup=/mydrive/yolov3' > data/obj.data

# `6.give image path and label`


In [11]:
#create a folder and unzip image dataset
!mkdir data/obj
!unzip /mydrive/yolov3/archive.zip -d data/obj
!unzip /mydrive/yolov3/images.zip -d data/Tobj


Archive:  /mydrive/yolov3/archive.zip
  inflating: data/obj/obj/0-with-mask.jpg  
  inflating: data/obj/obj/0-with-mask.txt  
  inflating: data/obj/obj/0.jpg      
  inflating: data/obj/obj/0.txt      
  inflating: data/obj/obj/1-with-mask.jpg  
  inflating: data/obj/obj/1-with-mask.txt  
  inflating: data/obj/obj/10-with-mask.jpg  
  inflating: data/obj/obj/10-with-mask.txt  
  inflating: data/obj/obj/100-with-mask.jpg  
  inflating: data/obj/obj/100-with-mask.txt  
  inflating: data/obj/obj/101-with-mask.jpg  
  inflating: data/obj/obj/101-with-mask.txt  
  inflating: data/obj/obj/103-with-mask.jpg  
  inflating: data/obj/obj/103-with-mask.txt  
  inflating: data/obj/obj/104-with-mask.jpg  
  inflating: data/obj/obj/104-with-mask.txt  
  inflating: data/obj/obj/105-with-mask.jpg  
  inflating: data/obj/obj/105-with-mask.txt  
  inflating: data/obj/obj/106-with-mask.jpg  
  inflating: data/obj/obj/106-with-mask.txt  
  inflating: data/obj/obj/107-with-mask.jpg  
  inflating: data/obj/

In [12]:
#Create train.txt file
import glob
images_list = glob.glob("data/obj/archive/*.jpg")
print(images_list)
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

    


[]


In [13]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)
with open("data/test.txt", "w") as f:
    f.write("\n".join(images_list))

    

[]


# 7.Transfer learning

In [14]:
# Download pre-trained weights for the convolutional layers file
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-06-04 05:09:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  45.9MB/s    in 3.7s    

2022-06-04 05:09:39 (41.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [15]:
# Start training

!./darknet detector train data/obj.data  cfg/yolov3_traning.cfg  darknet53.conv.74 -dont_show -mjpeg_port 8090 -map
#!./darknet detector train data/obj.data cfg/yolov3_traning.cfg   ../gdrive/MyDrive/yolov3/yolov3_traning_last.weights -dont_show 

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->